# 1. Import Package

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 500)
# import sys   
# sys.setrecursionlimit(25000)

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=64, progress_bar = True)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


 # 2. get all the urls, get df

In [2]:
news_url = 'https://www.hrw.org/sitesearch?search=&sort_by=created'
df = pd.DataFrame([news_url + '&page=%s'%i for i in range(0,2072+1)],  columns =['url'] )
df

,url
0,https://www.hrw.org/sitesearch?search=&sort_by...
1,https://www.hrw.org/sitesearch?search=&sort_by...
2,https://www.hrw.org/sitesearch?search=&sort_by...
3,https://www.hrw.org/sitesearch?search=&sort_by...
4,https://www.hrw.org/sitesearch?search=&sort_by...
...,...
2068,https://www.hrw.org/sitesearch?search=&sort_by...
2069,https://www.hrw.org/sitesearch?search=&sort_by...
2070,https://www.hrw.org/sitesearch?search=&sort_by...
2071,https://www.hrw.org/sitesearch?search=&sort_by...


In [3]:
def extract_urls(url):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    return [str(i) for i in soup.find_all('div', class_ = 'media-block__content')]

In [4]:
df[['articles']] = df['url'].parallel_apply(extract_urls)

In [5]:
df = df.explode('articles').reset_index(drop=True)
df

,url,articles
0,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
1,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
2,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
3,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
4,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
...,...,...
38788,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38789,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38790,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38791,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."


In [10]:
df.to_csv('hrw_raw_articles_more.csv',index= False)

# 3. Read saved df, start to handle df1

In [3]:
df1 = pd.read_csv('hrw_raw_articles_more.csv',header = [0])
df1

,url,articles
0,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
1,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
2,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
3,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
4,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
...,...,...
38788,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38789,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38790,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38791,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."


In [4]:
def extract_articles(article):
    
    article = BeautifulSoup(article, 'html.parser')
    
    headline = re.sub("^\s+|\s+$", "", article.a.text, flags=re.UNICODE) if article.a else ''
    short_story = re.sub("^\s+|\s+$", "", article.p.text, flags=re.UNICODE) if article.p else ''
    date = re.sub("^\s+|\s+$", "", article.time.text, flags=re.UNICODE) if article.time else ''

    link = 'https://www.hrw.org' + article.a['href']
    Type = re.sub("^\s+|\s+$", "", article.span.text, flags=re.UNICODE) if article.span else ''
    return pd.Series([headline, short_story, date, link, Type])

In [5]:
df1=df1[~df1.articles.isnull()].reset_index(drop=True)
df1

,url,articles
0,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
1,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
2,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
3,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
4,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
...,...,...
38648,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38649,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38650,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."
38651,https://www.hrw.org/sitesearch?search=&sort_by...,"<div class=""media-block__content"">\n<div class..."


In [9]:
pandarallel.initialize()
df1[['headline', 'short_story', 'date', 'link','Type']] = df1.articles.parallel_apply(extract_articles).values
df1['text'] = None
df1 = df1[['Type','headline','date','short_story', 'text','link']]
df1

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,Type,headline,date,short_story,text,link
0,News,Closing Camps Won’t Solve Kenya’s Refugee Problem,"April 13, 2021","Government Should Protect Refugees, Not Send P...",None,https://www.hrw.org/news/2021/04/13/closing-ca...
1,News,Letter to MFA in Ukraine Regarding Nationals A...,"April 12, 2021",,None,https://www.hrw.org/news/2021/04/12/letter-mfa...
2,News,North Korea’s Leader Warns of Famine,"April 12, 2021",Kim Jong Un May Be Tightening Already Firm Gri...,None,https://www.hrw.org/news/2021/04/12/north-kore...
3,News,Victims of Sudan Crackdown on Protests Await J...,"April 12, 2021","More Efforts Needed to Ensure Timely, Transpar...",None,https://www.hrw.org/news/2021/04/12/victims-su...
4,News,Brazil: Remove Miners from Indigenous Amazon T...,"April 12, 2021",Bolsonaro Policies Fuel Deforestation in Indig...,None,https://www.hrw.org/news/2021/04/12/brazil-rem...
...,...,...,...,...,...,...
38648,Report,Election 1984: Duvalier Style,"March 31, 1984",,None,https://www.hrw.org/report/1984/03/31/election...
38649,Report,Haiti: Report of a Human Rights Mission,"August 31, 1983",,None,https://www.hrw.org/report/1983/08/31/haiti-re...
38650,Report,Creating a Desolation and Calling it Peace,"May 1, 1983",,None,https://www.hrw.org/report/1983/05/01/creating...
38651,Report,Israeli Censorship of Arab Publications: A Survey,"January 1, 1983",,None,https://www.hrw.org/report/1983/01/01/israeli-...


In [13]:
df1=df1[~df1.duplicated()].reset_index(drop=True)
df1

,Type,headline,date,short_story,text,link
0,News,Closing Camps Won’t Solve Kenya’s Refugee Problem,"April 13, 2021","Government Should Protect Refugees, Not Send P...",None,https://www.hrw.org/news/2021/04/13/closing-ca...
1,News,Letter to MFA in Ukraine Regarding Nationals A...,"April 12, 2021",,None,https://www.hrw.org/news/2021/04/12/letter-mfa...
2,News,North Korea’s Leader Warns of Famine,"April 12, 2021",Kim Jong Un May Be Tightening Already Firm Gri...,None,https://www.hrw.org/news/2021/04/12/north-kore...
3,News,Victims of Sudan Crackdown on Protests Await J...,"April 12, 2021","More Efforts Needed to Ensure Timely, Transpar...",None,https://www.hrw.org/news/2021/04/12/victims-su...
4,News,Brazil: Remove Miners from Indigenous Amazon T...,"April 12, 2021",Bolsonaro Policies Fuel Deforestation in Indig...,None,https://www.hrw.org/news/2021/04/12/brazil-rem...
...,...,...,...,...,...,...
38640,Report,Election 1984: Duvalier Style,"March 31, 1984",,None,https://www.hrw.org/report/1984/03/31/election...
38641,Report,Haiti: Report of a Human Rights Mission,"August 31, 1983",,None,https://www.hrw.org/report/1983/08/31/haiti-re...
38642,Report,Creating a Desolation and Calling it Peace,"May 1, 1983",,None,https://www.hrw.org/report/1983/05/01/creating...
38643,Report,Israeli Censorship of Arab Publications: A Survey,"January 1, 1983",,None,https://www.hrw.org/report/1983/01/01/israeli-...


In [7]:
from newspaper import Article, Config
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def get_text_newspaper(url):
    try:
        a = Article(url, config= config)
        a.download()
        a.parse()
        text= a.text
        
        if len(text)>300:
            return pd.Series([text, 'text'])
        
        soup = BeautifulSoup(a.html, 'html.parser')
        try:
            body = link_soup.find_all('div', class_='article-body article-body--contained rich-text mx-auto')
            find_p = body[0].find_all('p')
            ptext = '\n'.join([p.text for p in find_p])
            if len(ptext)>300:
                return pd.Series([ptext, 'text'])
            return pd.Series([text, body[0]])
        except:
            return pd.Series([text, 'none'])
#             try:
#                 body = link_soup.find_all('article',{"role":"article"}')
#                 find_p = body[0].find_all('p')
#                 text = '\n'.join([p.text for p in find_p])
#                 return pd.Series([text, body[0]])
#             except: 
#                  body = soup.find_all('main', {"role":"main"})
#                  try:
#                      find_p = body[0].find_all('p')
#                      text = '\n'.join([p.text for p in find_p])
#                 except:
#                       text='error'
#                     return pd.Series([text, 'none']])                     
    except:
        text = 'error'
        print('error', url) 
    return text


def get_text_newspaper(url):
    try:
        a = Article(url, config= config)
        a.download()
        a.parse()
        text= a.text
        
        if len(text)>300:
            return pd.Series([text, None])
        else:      
            soup = BeautifulSoup(a.html, 'html.parser')           
            try:
                body = soup.find_all('div', class_='article-body article-body--contained rich-text mx-auto')
                find_p = body[0].find_all('p')
                ptext = '\n'.join([p.text for p in find_p])
                if len(ptext)>300:
                    return pd.Series([ptext, None])
                else:
                    return pd.Series([ptext, a.html])                    
            except:
                return pd.Series([text, a.html])
    except:
        text = 'error'
        print('error', url) 
    return pd.Series([text, 'error'])

In [2]:
df1 = pd.read_csv('df_temp.csv', header=0)
df1 = df1[~df1.link.str.contains('video-photos')].reset_index(drop=True)
df1

,Type,headline,date,short_story,text,link,html
0,News,Closing Camps Won’t Solve Kenya’s Refugee Problem,"April 13, 2021","Government Should Protect Refugees, Not Send People Back to Dangerous Conditions","Click to expand Image An aerial picture shows a section of the Hagadera camp in Dadaab near the Kenya-Somalia border, May 8, 2015. © 2015 Reuters\n\nThe Kenyan government has again issued an ultimatum to the United Nations High Commissioner for Refugees (UNHCR) to provide a plan and timeline for closing the Dadaab and Kakuma refugee camps. Though the government has issued these demands before, its new ultimatum has caused anxiety for more than 400,000 Somali, South Sudanese, Congolese, and o...",https://www.hrw.org/news/2021/04/13/closing-camps-wont-solve-kenyas-refugee-problem,NaN
1,News,Letter to MFA in Ukraine Regarding Nationals Arbitrarily Held in Syria,"April 12, 2021",NaN,"Ministry of Foreign Affairs of Ukraine\n\nMinister Dmytro Kuleba\n\nMykhailivska Square, 1, Kyiv, 01018\n\nformin@mfa.gov.ua\n\npublic.info@mfa.gov.ua\n\npress@mfa.gov.ua\n\nMarch 25, 2021\n\nRE: PLIGHT OF UKRAINIANS ARBITRARILY HELD IN NORTHEAST SYRIA\n\nDear Minister Kuleba, Human Rights Watch writes to call on your government to bring home or help repatriate, as a matter of urgent priority, the estimated 40 Ukrainian women, and children arbitrarily and indefinitely held in camps in northe...",https://www.hrw.org/news/2021/04/12/letter-mfa-ukraine-regarding-nationals-arbitrarily-held-syria,NaN
2,News,North Korea’s Leader Warns of Famine,"April 12, 2021",Kim Jong Un May Be Tightening Already Firm Grip on Power,"Click to expand Image North Korean leader Kim Jong Un delivers a closing speech at the Sixth Conference of Cell Secretaries of the Workers' Party of Korea in Pyongyang, North Korea, April 8, 2021. © 2021 Korean Central News Agency/Korea News Service via AP\n\nSpeaking at a ruling Workers’ Party of Korea conference on Thursday, North Korean leader Kim Jong Un was surprisingly candid about the country’s dire economic situation, calling on the country to “wage another more difficult ‘Arduous Ma...",https://www.hrw.org/news/2021/04/12/north-koreas-leader-warns-famine,NaN
3,News,Victims of Sudan Crackdown on Protests Await Justice,"April 12, 2021","More Efforts Needed to Ensure Timely, Transparent Accountability","Click to expand Image Sudanese celebrate after officials said the military had forced longtime autocratic President Omar al-Bashir to step down after 30 years in power in Khartoum, Sudan, Thursday, April 11, 2019. © 2019 AP Images\n\nWhen Sudan’s long-standing president Omar al-Bashir was ousted on April 11, 2019, protesters who had placed their lives on the line hoped this would mark the beginning of a freer and more just Sudan. Two years on, amidst slow progress and complex challenges, pro...",https://www.hrw.org/news/2021/04/12/victims-sudan-crackdown-protests-await-justice,NaN
4,News,Brazil: Remove Miners from Indigenous Amazon Territory,"April 12, 2021",Bolsonaro Policies Fuel Deforestation in Indigenous Lands,"(São Paulo, April 12, 2021) – Brazilian federal authorities should immediately remove miners who have unlawfully entered the Munduruku Indigenous territory in the Amazon rainforest, Human Rights Watch said today.\n\nMunduruku Indigenous people in the Tapajós basin – an epicenter of illegal gold mining in the Amazon rainforest – in southwestern Pará state have reported increasing encroachments upon their lands by armed “wildcat” miners known as “garimpeiros” since March 14, 2021. The Federal ...",https://www.hrw.org/news/2021/04/12/brazil-remove-miners-indigenous-amazon-territory,NaN
...,...,...,...,...,...,...,...
37171,Report,Election 1984: Duvalier Style,"March 31, 1984",NaN,"This 17- page report by the Americas Watch and the Lawyers Committee for International Human Rights finds that the circumstances in which the February 12, 1984 elections were held in Haiti involved, in the words of US Secretary of State George Shultz, 

In [5]:
df1[df1.text.isnull()]

,Type,headline,date,short_story,text,link,html
568,News,Top Human Rights Tweets of the Week,"January 22, 2021",NaN,NaN,https://www.hrw.org/news/2021/01/22/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
690,News,Top Human Rights Tweets of the Week,"January 10, 2021",NaN,NaN,https://www.hrw.org/news/2021/01/10/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
751,News,Trending Rights Tweets of 2020,"December 18, 2020",NaN,NaN,https://www.hrw.org/news/2020/12/18/trending-rights-tweets-2020,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
904,News,Top Human Rights Tweets of the Week,"December 6, 2020",NaN,NaN,https://www.hrw.org/news/2020/12/06/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
1013,News,Top Human Rights Tweets of the Week,"November 20, 2020",NaN,NaN,https://www.hrw.org/news/2020/11/21/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
...,...,...,...,...,...,...,...
19990,News,"Human Rights Watch Daily Brief, 26 June 2013","June 26, 2013","US and same-sex marriage, Egypt, Obama in Africa, Saudi Arabia",NaN,https://www.hrw.org/news/2013/06/26/human-rights-watch-daily-brief-26-june-2013,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
20038,News,"Human Rights Watch 

In [12]:
pandarallel.initialize(nb_workers=4)
batch_size = 4 * 4
i=resume = 21670
while i< df1.shape[0]:
    print(i, end=',')
    df1.loc[i: i+ batch_size-1, ['text','html']]=\
    df1.loc[i: i+ batch_size-1, 'link'].parallel_apply(get_text_newspaper).values
    i += batch_size
    if (i-resume)%(batch_size) == 10:
        df1.to_csv('df_temp.csv', index=False)
df1.to_csv('df_temp.csv', index=False)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
21670,21686,21702,21718,21734,21750,21766,21782,21798,21814,21830,21846,21862,error https://www.hrw.org/news/2012/06/14/afghanistan-reverse-suspension-political-party
error https://www.hrw.org/news/2012/06/15/syria-sexual-assault-detention
error https://www.hrw.org/news/2012/06/13/syria-and-icc
21878,21894,21910,21926,21942,21958,21974,21990,22006,22022,22038,22054,22070,22086,22102,22118,22134,22150,22166,22182,22198,22214,22230,error errorhttps://www.hrw.org/news/2012/04/09/chile-amend-and-pass-anti-discrimination-bill 
https://www.hrw.org/news/2012/04/09/chile-letter-congress-anti-discrimination-bill
error https://www.hrw.org/news/2012/04/10/bahrain-why-abdulhadi-al-khawaja-should-be-free
error https://www.hrw.org/news/2012/04/09/letter-secretary-defense-leon-panetta-repatriate-or-resettle-detainees-cleared
error https://www.hrw.org/news/2012/

In [8]:
pandarallel.initialize(nb_workers=2, progress_bar=True)
df1.loc[df1.html=='error', ['text','html']]=\
df1.loc[df1.html=='error', 'link'].parallel_apply(get_text_newspaper).values

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
errorerror  https://www.hrw.org/the-day-in-human-rights/2020/06/15https://www.hrw.org/news/2013/07/23/peru-ensure-fair-selection-judges-ombudsman

error https://www.hrw.org/news/2013/07/22/human-rights-watch-daily-brief-22-july
error https://www.hrw.org/news/2013/07/01/human-rights-watch-daily-brief-1-july
error https://www.hrw.org/news/2013/06/21/human-rights-watch-daily-brief-21-june
error https://www.hrw.org/news/2013/06/17/human-rights-watch-daily-brief-17-june
error https://www.hrw.org/news/2013/06/12/human-rights-watch-daily-brief-12-june
error https://www.hrw.org/news/2018/06/25/endfamilydetention-stop-trump-locking-families
error https://www.hrw.org/news/2017/10/06/rights-weekender
error https://www.hrw.org/news/2016/12/22/most-read-human-rights-stories-2016
error https://www.hrw.org/the-day-in-human-rights/2016/05/19
errorerror  https://

KeyboardInterrupt: 

In [5]:
def get_text_newspaper(url):
    try:
        time.sleep(1)
        a = Article(url, config= config)
        a.download()
        a.parse()
        text= a.text

    except:
        text = 'error'
        print('error', url) 
    return pd.Series([text, 'error'])

In [4]:
df2 = df1.loc[df1.html=='error'].copy().reset_index(drop=True)
df2

,Type,headline,date,short_story,text,link,html
0,Daily Brief,Huge Blow for Media Freedom in the Philippines: Daily Brief,"June 15, 2020",NaN,error,https://www.hrw.org/the-day-in-human-rights/2020/06/15,error
1,News,US: State Department Should Affirm Rights for All,"May 5, 2020",Unalienable Rights Panel Shouldn’t Rewrite International Law,error,https://www.hrw.org/news/2020/05/05/us-state-department-should-affirm-rights-all,error
2,Daily Brief,Millions in Syria Await Blocked Relief: Daily Brief,"April 29, 2020",NaN,error,https://www.hrw.org/the-day-in-human-rights/2020/04/29,error
3,Daily Brief,China Muzzling Covid Critics: Daily Brief,"April 28, 2020",NaN,error,https://www.hrw.org/the-day-in-human-rights/2020/04/28,error
4,News,ICC: Joint NGO Letter on State Party Support,"April 28, 2020",NaN,error,https://www.hrw.org/news/2020/04/28/icc-joint-ngo-letter-state-party-support,error
...,...,...,...,...,...,...,...
327,Report,Human Rights in Jamaica,"April 1, 1993",NaN,error,https://www.hrw.org/report/1993/04/01/human-rights-jamaica/death-penalty-prison-conditions-and-police-violence,error
328,Report,Human Rights in Peru: One Year After Fujimori’s Coup,"April 1, 1993",NaN,error,https://www.hrw.org/report/1993/04/01/human-rights-peru-one-year-after-fujimoris-coup,error
329,Report,No End in Sight,"April 1, 1993",NaN,error,https://www.hrw.org/report/1993/04/01/no-end-sight-human-rights-violations-assam,error
330,Report,Africa Watch Letters Protest Abuses of Human Rights by All Parties to the Conflict in Southern Sudan,"April 1, 1993",NaN,error,https://www.hrw.org/report/1993/04/01/africa-watch-letters-protest-abuses-human-rights-all-parties-conflict-southern,error


In [8]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options
# from icecream import ic
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import os

chrome_options = Options()
chrome_options.headless = False
chrome_drive = './chromedriver'
os.environ['webdrive.chrome.drive'] = chrome_drive

def get_rows(url):
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
    driver.get(url)
      
    html = driver.page_source
    time.sleep(2)
    driver.quit()

    a = Article('')
    a.set_html(html)
    a.parse()
    text= a.text

    if len(text)>300:
        return pd.Series([text, None])
    else:      
        soup = BeautifulSoup(a.html, 'html.parser')           
        try:
            body = soup.find_all('div', class_='article-body article-body--contained rich-text mx-auto')
            find_p = body[0].find_all('p')
            ptext = '\n'.join([p.text for p in find_p])
            if len(ptext)>300:
                return pd.Series([ptext, None])
            else:
                return pd.Series([ptext, a.html])                    
        except:
            return pd.Series([text, a.html])

In [9]:
pandarallel.initialize(nb_workers=8)
batch_size = 8
i=resume = 0
while i< df2.shape[0]:
    print(i, end=',')
    df2.loc[i: i+ batch_size-1, ['text','html']]=\
    df2.loc[i: i+ batch_size-1, 'link'].parallel_apply(get_rows).values
    i += batch_size

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,8,16,24,32,40,48,56,64,72,80,88,96,104,112,120,128,136,144,152,160,168,176,184,192,200,208,216,224,232,240,248,256,264,272,280,288,296,304,312,320,328,

In [11]:
df2

,Type,headline,date,short_story,text,link,html
0,Daily Brief,Huge Blow for Media Freedom in the Philippines: Daily Brief,"June 15, 2020",NaN,"Yesterday, a Manila court issued a guilty verdict for Maria Ressa, the founder and executive editor of the news website Rappler, and a Rappler researcher, Reynaldo Santos Jr. The verdict highlights the ability of the Philippines’ abusive leader to manipulate the laws to go after critical, well-respected media voices.\n\nA new report examines a range of defamation and incitement legal provisions that authorities in Iraq have used against critics, including journalists, activists, and other di...",https://www.hrw.org/the-day-in-human-rights/2020/06/15,None
1,News,US: State Department Should Affirm Rights for All,"May 5, 2020",Unalienable Rights Panel Shouldn’t Rewrite International Law,"Click to expand Image US Secretary of State Mike Pompeo, right, unveils the creation of Commission on ""Unalienable"" Rights, headed by Mary Ann Glendon, left, a Harvard Law School professor and a former U.S. Ambassador to the Holy See, during an announcement at the US State Department in Washington, July 8, 2019. © 2019 AP Photo/Pablo Martinez Monsivais\n\nThe United States State Department should reject any attempt by its Commission on Unalienable Rights to rewrite international law by rejec...",https://www.hrw.org/news/2020/05/05/us-state-department-should-affirm-rights-all,None
2,Daily Brief,Millions in Syria Await Blocked Relief: Daily Brief,"April 29, 2020",NaN,"Restrictions on aid deliveries from Damascus and Iraq are preventing medical supplies and personnel needed to prevent, contain, and treat Covid-19 from reaching two million people in northeast Syria.\n\nGökhan Türkmen, is one of at least two dozen people over the past three years reportedly abducted and forcibly disappeared by Turkish government agents for many months.\n\nThe Australian government cannot ignore how some Southeast Asian countries are using the Covid-19 pandemic to repress rig...",https://www.hrw.org/the-day-in-human-rights/2020/04/29,None
3,Daily Brief,China Muzzling Covid Critics: Daily Brief,"April 28, 2020",NaN,"While Beijing is ramping up its global propaganda about its “achievement in the fight against Covid-19”, the Chinese government is arbitrarily detaining activists and forcibly disappearing citizen journalists for sharing critical information about the pandemic.\n\nAfghan women and girls with disabilities are frequently socially isolated and face discrimination and sexual harassment in accessing government assistance, health care, and schools. The Covid-19 crisis exacerbates the problems.\n\n...",https://www.hrw.org/the-day-in-human-rights/2020/04/28,None
4,News,ICC: Joint NGO Letter on State Party Support,"April 28, 2020",NaN,"New York, 28 April 2020\n\nYour Excellency,\n\nWe, the undersigned organizations, call on your State and all States Parties to the Rome Statute to show support for the Rome Statute system and the International Criminal Court (ICC), especially in the face of threats to its independence and mandate. Although threats to the pursuit of international justice, including in the context of the ICC, are not new, protecting the ICC is particularly important today in the face of escalating hostility to...",https://www.hrw.org/news/2020/04/28/icc-joint-ngo-letter-state-party-support,None
...,...,...,...,...,...,...,...
327,Report,Human Rights in Jamaica,"April 1, 1993",NaN,"This report concerns the application of the death penalty, the conditions in prisons and lockups, and police violence, including acts of coercion to obtain evidence that amount to torture and the excessive use of deadly force.",https://www.hrw.org/report/1993/04/01/human-rights-jamaica/death-penalty-prison-conditions-and-police-violence,"<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000

In [15]:
df1.loc[df1.html=='error',['text','html']] = df2[['text','html']].values
df1

,Type,headline,date,short_story,text,link,html
0,News,Closing Camps Won’t Solve Kenya’s Refugee Problem,"April 13, 2021","Government Should Protect Refugees, Not Send People Back to Dangerous Conditions","Click to expand Image An aerial picture shows a section of the Hagadera camp in Dadaab near the Kenya-Somalia border, May 8, 2015. © 2015 Reuters\n\nThe Kenyan government has again issued an ultimatum to the United Nations High Commissioner for Refugees (UNHCR) to provide a plan and timeline for closing the Dadaab and Kakuma refugee camps. Though the government has issued these demands before, its new ultimatum has caused anxiety for more than 400,000 Somali, South Sudanese, Congolese, and o...",https://www.hrw.org/news/2021/04/13/closing-camps-wont-solve-kenyas-refugee-problem,NaN
1,News,Letter to MFA in Ukraine Regarding Nationals Arbitrarily Held in Syria,"April 12, 2021",NaN,"Ministry of Foreign Affairs of Ukraine\n\nMinister Dmytro Kuleba\n\nMykhailivska Square, 1, Kyiv, 01018\n\nformin@mfa.gov.ua\n\npublic.info@mfa.gov.ua\n\npress@mfa.gov.ua\n\nMarch 25, 2021\n\nRE: PLIGHT OF UKRAINIANS ARBITRARILY HELD IN NORTHEAST SYRIA\n\nDear Minister Kuleba, Human Rights Watch writes to call on your government to bring home or help repatriate, as a matter of urgent priority, the estimated 40 Ukrainian women, and children arbitrarily and indefinitely held in camps in northe...",https://www.hrw.org/news/2021/04/12/letter-mfa-ukraine-regarding-nationals-arbitrarily-held-syria,NaN
2,News,North Korea’s Leader Warns of Famine,"April 12, 2021",Kim Jong Un May Be Tightening Already Firm Grip on Power,"Click to expand Image North Korean leader Kim Jong Un delivers a closing speech at the Sixth Conference of Cell Secretaries of the Workers' Party of Korea in Pyongyang, North Korea, April 8, 2021. © 2021 Korean Central News Agency/Korea News Service via AP\n\nSpeaking at a ruling Workers’ Party of Korea conference on Thursday, North Korean leader Kim Jong Un was surprisingly candid about the country’s dire economic situation, calling on the country to “wage another more difficult ‘Arduous Ma...",https://www.hrw.org/news/2021/04/12/north-koreas-leader-warns-famine,NaN
3,News,Victims of Sudan Crackdown on Protests Await Justice,"April 12, 2021","More Efforts Needed to Ensure Timely, Transparent Accountability","Click to expand Image Sudanese celebrate after officials said the military had forced longtime autocratic President Omar al-Bashir to step down after 30 years in power in Khartoum, Sudan, Thursday, April 11, 2019. © 2019 AP Images\n\nWhen Sudan’s long-standing president Omar al-Bashir was ousted on April 11, 2019, protesters who had placed their lives on the line hoped this would mark the beginning of a freer and more just Sudan. Two years on, amidst slow progress and complex challenges, pro...",https://www.hrw.org/news/2021/04/12/victims-sudan-crackdown-protests-await-justice,NaN
4,News,Brazil: Remove Miners from Indigenous Amazon Territory,"April 12, 2021",Bolsonaro Policies Fuel Deforestation in Indigenous Lands,"(São Paulo, April 12, 2021) – Brazilian federal authorities should immediately remove miners who have unlawfully entered the Munduruku Indigenous territory in the Amazon rainforest, Human Rights Watch said today.\n\nMunduruku Indigenous people in the Tapajós basin – an epicenter of illegal gold mining in the Amazon rainforest – in southwestern Pará state have reported increasing encroachments upon their lands by armed “wildcat” miners known as “garimpeiros” since March 14, 2021. The Federal ...",https://www.hrw.org/news/2021/04/12/brazil-remove-miners-indigenous-amazon-territory,NaN
...,...,...,...,...,...,...,...
37171,Report,Election 1984: Duvalier Style,"March 31, 1984",NaN,"This 17- page report by the Americas Watch and the Lawyers Committee for International Human Rights finds that the circumstances in which the February 12, 1984 elections were held in Haiti involved, in the words of US Secretary of State George Shultz, 

In [16]:
df1.loc[df1.html=='error']

,Type,headline,date,short_story,text,link,html


In [17]:
df1.to_csv('df_temp_1.csv', index=False)

In [22]:
df1.loc[df1.text.isnull()]

,Type,headline,date,short_story,text,link,html
568,News,Top Human Rights Tweets of the Week,"January 22, 2021",NaN,NaN,https://www.hrw.org/news/2021/01/22/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
690,News,Top Human Rights Tweets of the Week,"January 10, 2021",NaN,NaN,https://www.hrw.org/news/2021/01/10/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
751,News,Trending Rights Tweets of 2020,"December 18, 2020",NaN,NaN,https://www.hrw.org/news/2020/12/18/trending-rights-tweets-2020,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
904,News,Top Human Rights Tweets of the Week,"December 6, 2020",NaN,NaN,https://www.hrw.org/news/2020/12/06/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
1013,News,Top Human Rights Tweets of the Week,"November 20, 2020",NaN,NaN,https://www.hrw.org/news/2020/11/21/top-human-rights-tweets-week,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
...,...,...,...,...,...,...,...
19990,News,"Human Rights Watch Daily Brief, 26 June 2013","June 26, 2013","US and same-sex marriage, Egypt, Obama in Africa, Saudi Arabia",NaN,https://www.hrw.org/news/2013/06/26/human-rights-watch-daily-brief-26-june-2013,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
20038,News,"Human Rights Watch 

In [31]:
idd = df1[df1.text.isnull() & df1.link.str.contains('tweets')].index
idd.shape

(153,)

In [32]:
df1 = df1[~df1.index.isin(idd)].reset_index(drop=True)

In [33]:
df1[df1.text.isnull()]

,Type,headline,date,short_story,text,link,html
3232,World Report Chapter,World Report 2020: Live from the United Nations,"January 14, 2020",NaN,NaN,https://www.hrw.org/world-report/2020/live-press-conference,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
3756,News,Australian Health Practitioner Regulation Agency Response to HRW,"October 25, 2019",NaN,NaN,https://www.hrw.org/news/2019/10/25/australian-health-practitioner-regulation-agency-response-hrw,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
4050,News,Iran’s ‘Blue Girl’ Suicide Sparks Renewed Calls on FIFA Over Stadium Ban,"September 10, 2019",NaN,NaN,https://www.hrw.org/news/2019/09/10/irans-blue-girl-suicide-sparks-renewed-calls-fifa-over-stadium-ban,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
4211,News,Social Care in the United Kingdom,"August 13, 2019",NaN,NaN,https://www.hrw.org/news/2019/08/13/social-care-united-kingdom,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head><link rel='dns-prefetch' href='fast.fonts.net'><link rel='dns-prefetch..."
4658,News,Letter Response from DEFRA to Human Rights Watch,"June 11, 2019",NaN,NaN,https://www.hrw.org/news/2019/06/11/letter-response-defra-human-rights-watch,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# "">\n <head>\n <meta charset=""utf-8"" />\n<script>window.dataLayer = window.data..."
...,...,...,...,...,...,...,...
19837,News,"Human Rights Watch Daily Brief, 26 June 2013","June 26, 2013","US and same-sex marriage, Egypt, Obama in Africa, Saudi Arabia",NaN,https://www.hrw.org/news/2013/06/26/human-rights-watch-daily-brief-26-june-2013,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" prefix=""content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://

In [62]:
# df1.loc[df1.text.isnull(),'text']='None'
# df1[ (df1.text.str.len()<200) & (df1.text.str.len()>100)]

# df1[ (df1.text.str.len()<200) & (df1.text.str.len()>100)]

# df1 = df1[~df1.text.str.contains('JavaScript is')]
# df1= df1.reset_index(drop=True)
# df1[(df1.text.str.len()<100)]
# df1 = df1[(df1.text.str.len()>200)]
# df1.text = df1.text.lambda(x: x.replace('Click to expand Image',''))
df1.loc[df1.text.str.startswith('Click to expand Image'),'text'] =\
df1.loc[df1.text.str.startswith('Click to expand Image'),'text'].apply(lambda x: x.replace('Click to expand Image',''))

In [76]:
# df1 = df1[~df1.text.duplicated()] 
# df1[df1.link.duplicated()]
df2 = df1.iloc[:,:6]
df2.columns=['Type',	'title',	'date',	'abstract',	'text', 'url']
df2= df2.reset_index(drop=True)
df2.date = pd.to_datetime(df2['date'])
df2.sort_values('date')

,Type,title,date,abstract,text,url
36376,Report,Israeli Censorship of Arab Publications: A Survey,1983-01-01,NaN,"In its pre-publication review of Palestinian newspapers and magazines, Israel’s military censor blocks the publication not only of supposed national security secrets or material likely to incite violence. Rather, the censor’s primary concern, as shown through a study of banned materials, is to eradicate expression that could foster Palestinian nationalist feelings; suggests Palestinians are a nation with a national heritage; denies Israel’s legitimacy; or imputes behavior to Israeli official...",https://www.hrw.org/report/1983/01/01/israeli-censorship-arab-publications-survey/fund-free-expression-report-meron
36375,Report,Haiti: Report of a Human Rights Mission,1983-08-31,NaN,"This 24 page report by the Lawyers Committee for International Human Rights, Americas Watch Committee, and the International League for Human Rights examines the situation of people who support human rights in Haiti, and in particular a series of detentions in May 1983.\n\nIn recent years, Haitian authorities have frequently seized persons they consider disloyal to the goverment, detained them incommunicado without formal charges, and then either released them without explanation or subjecte...",https://www.hrw.org/report/1983/08/31/haiti-report-human-rights-mission
36374,Report,Election 1984: Duvalier Style,1984-03-31,NaN,"This 17- page report by the Americas Watch and the Lawyers Committee for International Human Rights finds that the circumstances in which the February 12, 1984 elections were held in Haiti involved, in the words of US Secretary of State George Shultz, a denial of ""all the preliminary aspects that make an election really mean something.""",https://www.hrw.org/report/1984/03/31/election-1984-duvalier-style/report-human-rights-haiti-based-mission-inquiry
36373,Report,Journalists in Jeopardy,1984-10-11,NaN,"The Committee to Protect Journalists and the the Americas Watch mission to Haiti from August 12-15,1984 to investigate a recent crackdown on the press. This 12-page documents the ways in which, in both law and practice, Haitian journalists are routinelly prohibited from criticizing their country's ""institutions."" Thus, the newspapers and radio stations that criticize the most durable Haitian institutions of all- poverty and inequality- are the most vulnerable to goverment oppression.",https://www.hrw.org/report/1984/10/11/journalists-jeopardy/haitian-reality
36372,Report,Rights Denied,1985-03-31,NaN,"This 25 page report by the National Coalition for Haitian Refugees, Americas Watch, and the Lawyers Committee for International Human Rights examines human rights conditions in 1984, a year when respect for human rights continued to be poor, and in some respects deteriorated significantly.",https://www.hrw.org/report/1985/03/31/rights-denied/report-human-rights-haiti-1984
...,...,...,...,...,...,...
4,News,Brazil: Remove Miners from Indigenous Amazon Territory,2021-04-12,Bolsonaro Policies Fuel Deforestation in Indigenous Lands,"(São Paulo, April 12, 2021) – Brazilian federal authorities should immediately remove miners who have unlawfully entered the Munduruku Indigenous territory in the Amazon rainforest, Human Rights Watch said today.\n\nMunduruku Indigenous people in the Tapajós basin – an epicenter of illegal gold mining in the Amazon rainforest – in southwestern Pará state have reported increasing encroachments upon their lands by armed “wildcat” miners known as “garimpeiros” since March 14, 2021. The Federal ...",https://www.hrw.org/news/2021/04/12/brazil-remove-miners-indigenous-amazon-territory
3,News,Victims of Sudan Crackdown on Protests Await Justice,2021-04-12,"More Efforts Needed to Ensure Timely, Transparent Accountability","Sudanese celebrate after officials said the military had forced longtime autocratic President Omar al-Bashir to step down after 30 years in power in Khartoum, Sudan, Thursday, April 11, 20

In [77]:
df2.to_csv('hrw.csv', index=False)

In [78]:
df_sample = df2.sample(n=50).reset_index(drop=True)
for i,j in zip(df_sample.url, df_sample.text):
    print(i)
    print(j)
    print('\n')

https://www.hrw.org/news/2011/07/06/mexico-ruling-affirms-obligation-military-justice-reform
(Washington, DC) - A ruling on July 6, 2011, by Mexico's Supreme Court should bring reform of Mexico's flawed military justice system and accountability for soldiers accused of human rights violations closer, Human Rights Watch said today. The court ruled that Mexico's courts are obligated to comply with an Inter-American Court judgment in an enforced disappearance case and that its jurisprudence should be taken into account by Mexico's judges.

In a binding November 2009 judgment, Radilla Pacheco v. Mexico, the Inter-American Court of Human Rights found that the Mexican military was responsible for the disappearance of Rosendo Radilla Pacheco in Guerrero state in 1974 and that the government had failed to investigate the crime adequately. The Inter-American Court ordered Mexico to modify its Code of Military Justice to make it "compatible with international standards," and said that, "Regardin